In [ ]:
import re
import random

data_path = "/content/English.txt"
data_path2 = "/content/Hindi.txt"

# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().strip().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
    lines2 = f.read().strip().split('\n')

lines = [" ".join(re.findall(r"[A-Za-z0-9]+", line)) for line in lines]
lines2 = [re.sub(r"%s|\(|\)|<|>|%|[a-z]|[A-Z]|_", '', line) for line in lines2]

# Grouping lines by response pair
data = list(zip(lines, lines2))
pairs = []

for pair in data:
    pairs.append('{{ "en": "{}", "hi": "{}" }}'.format(pair[0], pair[1]))

random.shuffle(data)

pairs


['{ "en": "new jersey is sometimes quiet during autumn and it is snowy in april", "hi": "नई जर्सी कभी-कभी शरद ऋतु के दौरान शांत होती है, और यह अप्रैल में बर्फीली होती है।" }',
 '{ "en": "the united states is usually chilly during july and it is usually freezing in november", "hi": "संयुक्त राज्य आमतौर पर जुलाई के दौरान मिर्च होता है, और यह आमतौर पर नवम्बर में जम जाता है।" }',
 '{ "en": "california is usually quiet during march and it is usually hot in june", "hi": "कैलिफ़ोर्निया आमतौर पर मार्च के दौरान शांत होता है, और आमतौर पर जून में गर्म होता है।" }',
 '{ "en": "the united states is sometimes mild during june and it is cold in september", "hi": "संयुक्त राज्य कभी-कभी जून के दौरान हल्के होते हैं, और यह सितंबर में ठंडा होता है।" }',
 '{ "en": "your least liked fruit is the grape but my least liked is the apple", "hi": "आपका सबसे कम पसंद किया जाने वाला फल अंगूर है, लेकिन मेरा सबसे कम पसंद किया जाने वाला सेब है।" }',
 '{ "en": "his favorite fruit is the orange but my favorite is the gra

In [ ]:
pairs

['{ "en": "new jersey is sometimes quiet during autumn and it is snowy in april", "hi": "नई जर्सी कभी-कभी शरद ऋतु के दौरान शांत होती है, और यह अप्रैल में बर्फीली होती है।" }',
 '{ "en": "the united states is usually chilly during july and it is usually freezing in november", "hi": "संयुक्त राज्य आमतौर पर जुलाई के दौरान मिर्च होता है, और यह आमतौर पर नवम्बर में जम जाता है।" }',
 '{ "en": "california is usually quiet during march and it is usually hot in june", "hi": "कैलिफ़ोर्निया आमतौर पर मार्च के दौरान शांत होता है, और आमतौर पर जून में गर्म होता है।" }',
 '{ "en": "the united states is sometimes mild during june and it is cold in september", "hi": "संयुक्त राज्य कभी-कभी जून के दौरान हल्के होते हैं, और यह सितंबर में ठंडा होता है।" }',
 '{ "en": "your least liked fruit is the grape but my least liked is the apple", "hi": "आपका सबसे कम पसंद किया जाने वाला फल अंगूर है, लेकिन मेरा सबसे कम पसंद किया जाने वाला सेब है।" }',
 '{ "en": "his favorite fruit is the orange but my favorite is the gra

In [ ]:
pairs

[('her most loved fruit is the mango but my most loved is the lime',
  'उसका सबसे प्रिय फल आम है, लेकिन मेरा सबसे प्रिय फल चूना है।'),
 ('my most loved animal was that bird', 'मेरा सबसे प्रिय जानवर वह पक्षी था।'),
 ('california is sometimes cold during june and it is sometimes hot in december',
  'कैलिफोरनिया कभी-कभी जून के दौरान ठंडा होता है, और कभी-कभी हिरण में गर्म होता है।'),
 ('france is usually chilly during december and it is never wet in march',
  'फ्रेंस आमतौर पर डेली के दौरान मिर्च होती है, और यह मार्च में कभी गीला नहीं होता है।'),
 ('new jersey is sometimes mild during summer and it is usually hot in march',
  'नई जर्सी गर्मियों के दौरान कभी-कभी हल्की होती है, और आमतौर पर मार्च में गर्म होती है।'),
 ('paris is usually freezing during october and it is usually nice in april',
  'पेरिस के दौरान पेरिस आमतौर पर जम जाता है, और यह आमतौर पर अप्रैल में अच्छा होता है।'),
 ('france is sometimes wet during june but it is never pleasant in autumn',
  'फ्यून कभी-कभी जून के दौरान गीला होत

We will have to create separate lists for input sequences and target sequences and we will also need to create lists for unique tokens (input tokens and target tokens) in our dataset. For target sequences, we will add ‘’ at the beginning of the sequence and ‘’ at the end of the sequence so that our model knows where to start and end text generation.

In [ ]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation
  #target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)

  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split(" "):
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)


we convert both input tokens and output tokens as key value pairs . To decode the sentences we will need to create the reverse features dictionary that stores index as a key and word as a value.

In [ ]:

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])
reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())

encoding


In [ ]:
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.

    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from keras.layers import LeakyReLU
leaky_relu = LeakyReLU(alpha=0.2)
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 256
epochs = 20
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True,activation='relu')
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
history=training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('training_model.h5')

Epoch 1/20
32/32 [==============================] - 16s 338ms/step - loss: 0.9836 - accuracy: 0.0284 - val_loss: 0.8507 - val_accuracy: 0.0108
Epoch 2/20
32/32 [==============================] - 4s 135ms/step - loss: 0.8342 - accuracy: 0.0133 - val_loss: 0.8269 - val_accuracy: 0.0150
Epoch 3/20
32/32 [==============================] - 5s 161ms/step - loss: 0.8101 - accuracy: 0.0183 - val_loss: 0.8077 - val_accuracy: 0.0196
Epoch 4/20
32/32 [==============================] - 5s 153ms/step - loss: 0.7966 - accuracy: 0.0216 - val_loss: 0.7993 - val_accuracy: 0.0228
Epoch 5/20
32/32 [==============================] - 5s 151ms/step - loss: 0.7847 - accuracy: 0.0239 - val_loss: 0.7837 - val_accuracy: 0.0237
Epoch 6/20
32/32 [==============================] - 6s 179ms/step - loss: 0.7712 - accuracy: 0.0247 - val_loss: 0.7728 - val_accuracy: 0.0231
Epoch 7/20
32/32 [==============================] - 4s 138ms/step - loss: 0.7531 - accuracy: 0.0266 - val_loss: 0.7529 - val_accuracy: 0.0266
Epoch

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.savefig('wordA.png')
plt.figure()
plt.show()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'o', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.savefig('wordL.png')
plt.figure()
plt.show()

testing setup

In [ ]:
from tensorflow import keras
from keras.models import load_model
from keras.layers import Input, LSTM, Dense
from keras.models import Model
training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

testing the model

In [ ]:
def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.

    #A variable to store our response word by word
    decoded_sentence = ''

    stop_condition = False
    while not stop_condition:
      #Predicting output tokens with probabilities and states
      output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
#Choosing the one with highest probability
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_token = reverse_target_features_dict[sampled_token_index]
      decoded_sentence += " " + sampled_token
#Stop if hit max length or found the stop token
      if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
        stop_condition = True
#Update the target sequence
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.
      #Update states
      states_value = [hidden_state, cell_state]
    return decoded_sentence

In [ ]:
class Translator:
  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")

  #Method to start the translator
  def start(self):
    user_response = input("Give in an English sentence. :) \n")
    self.translate(user_response)

  #Method to handle the conversation
  def translate(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")

  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
#---------------------------------------------------------------------------------------
  # def string_to_matrix(self, user_input):
  #   # Tokenize the user input
  #   tokens = re.findall(r"[\w']+|[^\s\w]", user_input)

  #   # Initialize the user input matrix
  #   user_input_matrix = np.zeros(
  #       (1, max_encoder_seq_length, num_encoder_tokens),
  #       dtype='float32')

  #   # Debugging: Print tokens to check tokenization
  #   print("Tokens:", tokens)

  #   # Iterate over tokens and encode them in the matrix
  #   for timestep, token in enumerate(tokens):
  #       if token in input_features_dict:
  #           token_index = input_features_dict[token]
  #           if token_index < num_encoder_tokens:  # Ensure index is within bounds
  #               user_input_matrix[0, timestep, token_index] = 1.
  #           else:
  #               print("Warning: Token index out of bounds:", token_index)
  #       else:
  #           print("Warning: Token not found in dictionary:", token)

  #   return user_input_matrix
  #--------------------------------------------------------------------------------------
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response

  #Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
    return False

translator = Translator()

In [ ]:
translator.start()